In [1]:
import pygame as pg
import pygame.draw as dw
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

pygame 2.6.0 (SDL 2.28.4, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


# RK4
Do tipo
$$\frac{dr}{dt} = f(x, t)$$

Para uma eq de movimento podemos ter $\frac{d^2\vec r}{dt^2}=f(\vec r,\frac{d\vec r}{dt}, t)$, que podemos resolver como:
$$\begin{cases}
\frac{d\vec r}{dt}=\vec v\\
\frac{d\vec v}{dt}= f(\vec r,\vec v,t)
\end{cases}$$

In [ ]:
# Aqui, a forma como se coloca r na função f() depende completamente de como f está definida e funciona

def RK4_2var(f, xi, yi, a, b, N):
    x, y = np.zeros(N), np.zeros(N)
    x[0] = xi
    y[0] = yi
    r = np.array([x,y]).T # transposta para cada linha ser um ponto (x,y)
    h = (b-a)/N
    t = np.linspace(a,b,N)
    for i in range(1, N):
        k1 = h * f(*r[i-1], t[i-1])
        k2 = h * f(*r[i-1] + 0.5*k1, t[i-1] + 0.5*h)
        k3 = h * f(*r[i-1] + 0.5*k2, t[i-1] + 0.5*h)
        k4 = h * f(*r[i-1] + k3, t[i-1] + h)

        r[i] = r[i-1] + 1/6*(k1 + 2*k2 + 2*k3 + k4)
    
    return (r.T), t

# Caixa com Gravidade apenas

$$f = \vec a = (0,10)$$

In [2]:
def animate():
    L = 1000
    disp = pg.display.set_mode((L,L), pg.RESIZABLE)
    pg.display.set_caption("Teste #1")
    clk = pg.time.Clock()
    FPS = 60

    # centro e raio de circulo
    C = np.array([L/2,L/2])
    Wc = 10 # espessura do circulo em px
    Rc = L/12*5 - Wc
    # bola
    R = L/100
    r = C + np.array([10,-10])
    v = np.array([50,-50])
    sol = np.array([r[0], r[1], v[0], v[1]])
    dw.circle(disp, (255,255,255), C, Rc, 10)

    a = np.array([0,-10]) 

    colorTraj = [100,100,100]
    points = [(r[0],r[1])]

    m = 1 # massa (kg)
    
    N = 500
    dt = 100/N
    
    e = 0.5

    rotMat = lambda th: np.array([[np.cos(th),-np.sin(th)],[np.sin(th),np.cos(th)]])

    def f(x,y,vx,vy,t):
        fx = 0
        fy = 10
        return np.array([vx, vy, fx, fy], float)
        

    for i in range(N):
        for event in pg.event.get():
            if event.type == pg.QUIT:
                return None
            if event.type == pg.VIDEORESIZE:
                disp = pg.display.set_mode((event.w, event.h),pg.RESIZABLE)
                L = np.min([event.h, event.w])
                yL = 0.75*L
        disp.fill((0,0,0)) # refresh fundo da pagina

        dw.circle(disp, (255,255,255), C, Rc, 10)

        
        t = i*dt
        k1 = dt * f(*sol, t)
        k2 = dt * f(*sol + 0.5*k1, t + 0.5*dt)
        k3 = dt * f(*sol + 0.5*k2, t + 0.5*dt)
        k4 = dt * f(*sol + k3, t + dt)

        sol = sol + 1/6*(k1 + 2*k2 + 2*k3 + k4)
        r = sol[:2]
        v = sol[2:]
        # paredes circulo
        if np.linalg.norm(r-C) >= Rc - 2*R :
            th = np.arctan2(v[0], v[1])
            beta = np.arctan2((r-C)[1], (r-C)[0])
            angIn = th + beta
            matriz = rotMat(2*angIn) # matriz de rotação
            v = matriz @ v # rodar velocidade
            v *= np.sqrt(e) # atenuação da energia cinética por um fator e
            r = C + (Rc - 2*R) * (r-C)/np.linalg.norm(r-C)
            
        sol = np.array([r[0], r[1], v[0], v[1]])

        points.append((r[0],r[1]))
        if i % 3 == 0: # 0,3,6,9 
            if colorTraj[0] + 2 > 255: colorTraj[0] = 0
            colorTraj[0] += 2
        if i-1 % 3 == 0: # 1,4,7,10
            if colorTraj[1] + 4 > 255: colorTraj[1] = 0
            colorTraj[1] += 4
        if i-2 % 3 == 0: # 2,5,8,11
            if colorTraj[2] - 2 < 0: colorTraj[2] = 255
            colorTraj[2] -= 2
        dw.lines(disp, colorTraj, False, points, 2)
        dw.circle(disp, (255,255,255), (r[0], r[1]), R, 0)
        pg.display.update()
        clk.tick(FPS)
        x3 = pg.surfarray.pixels3d(disp)
        array = np.uint8(x3)
        im = Image.fromarray(array)
        im = im.save(f'./photos/{i:04d}.png', 'PNG')



animate()
pg.quit()

